In [1]:
import easyocr
import time
import cv2
import requests
import base64
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

reader = easyocr.Reader(['th', 'en'])

In [2]:
import mysql.connector

mydb = mysql.connector.connect(
    host="209.15.113.245",
    user="tma",
    password="Amazon1234!",
    database="calories"
)

print(mydb)

In [4]:
mycursor = mydb.cursor()
mycursor.execute("""SELECT sum(case when cal_img like 'https://caloriescredit.tmadigital.com/uploads/calories_img/%' then 1 else 0 end) as tma, 
                 sum(case when cal_img like 'https://ccc.mots.go.th/uploads/calories_img/%' then 1 else 0 end) as ccc, 
                 count(*) as total 
                 FROM `cal_customer_send_activity_detail` WHERE cs_id>6968796""")
myresult = mycursor.fetchall()
for x in myresult:
    print(x)

(Decimal('2353'), Decimal('252523'), 887413)


In [5]:
sql = """select cs_id,time_stamp,cal_img
FROM `cal_customer_send_activity_detail` 
WHERE cs_id>6968796 AND (cal_img LIKE 'https://ccc.mots.go.th/uploads/calories_img/%'
OR cal_img LIKE 'https://caloriescredit.tmadigital.com/uploads/calories_img/%');"""
mycursor.execute(sql)
myresult = mycursor.fetchall()

In [6]:
i=0
total=len(myresult)
start = time.time()

for x in myresult:
   url=x[2]
   x1 = url.split("/")
   filename = x1[-1]
   #print(filename)
   start_get = time.time()
   try:
      resp = requests.get(url)

      image = np.asarray(bytearray(resp.content), dtype="uint8")
      image = cv2.imdecode(image, cv2.IMREAD_COLOR)
      image_ori = image.copy()
      #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

      result = reader.readtext(image)
      points=[]
      text=[]
      for d in result:
            text.append(d[1])
            if ":" in d[1] or any(char.isdigit() for char in d[1]):
               points.append([[int(d[0][0][0]),int(d[0][0][1])],[int(d[0][2][0]),int(d[0][2][1])],d[1]])
               image = cv2.rectangle(image, [int(d[0][0][0]),int(d[0][0][1])], [int(d[0][2][0]),int(d[0][2][1])], (255, 0, 0), 2)
      _, im_arr = cv2.imencode('.jpg', image)
      im_bytes = im_arr.tobytes()
      im_b64 = base64.b64encode(im_bytes)
      if len(points)<2:
         cv2.imwrite("images/nok/"+filename, image)
         cv2.imwrite("original_images/nok/"+filename, image_ori)
         sql = "INSERT INTO image_process(cs_id, filename, process_time,is_ok,text_list) VALUES (%s, %s, %s, %s, %s)"
         values = (x[0], filename, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),"N",str(text).replace("'",'"'))
         mycursor.execute(sql,values)
         mydb.commit()
      else:
         cv2.imwrite("images/ok/"+filename, image)
         cv2.imwrite("original_images/ok/"+filename, image_ori)
         sql = "INSERT INTO image_process(cs_id, filename, process_time,is_ok,text_list) VALUES (%s, %s, %s, %s, %s)"
         values = (x[0], filename, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),"Y",str(text).replace("'",'"'))
         mycursor.execute(sql,values)
         mydb.commit()
      #plt.imshow(image)
      #plt.axis('off')  # Hide axes for a cleaner look
      #plt.show()
      i+=1
      if i%100==0:
         print(i,"of",total)
      #if i%(total//100)==0:
      #   end = time.time()
      #   print(end-start, i, i/(total//100)," %")
   except Exception as inst:
      print("exception:",inst,x[0])
      continue
end = time.time()
print("Total image:", total)
print("Total Time:", end-start)

100 of 254876
200 of 254876
300 of 254876
400 of 254876
500 of 254876
600 of 254876
700 of 254876
800 of 254876
900 of 254876
1000 of 254876
1100 of 254876
1200 of 254876
1300 of 254876
1400 of 254876
1500 of 254876
1600 of 254876
1700 of 254876
1800 of 254876


KeyboardInterrupt: 